In [1]:
from datetime import datetime
def time_now():
    '''Get Current Time'''
    
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S.%f")
    print("Current Time =", current_time)
    return now


############
print('Starting...')
start = time_now()
############


Starting...
Current Time = 17:22:05.119624


In [18]:
import swifter

In [2]:
import pandas as pd

In [3]:
import regex as re

In [4]:
from itertools import chain

In [5]:
from collections import Counter

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

from spacy.matcher import Matcher

In [7]:
import os

In [8]:
ignore_list = ['#', '’', "'"  '!',  ',',  '(',  '.',  '‡',  '9',  '*',  '2',  ']',  '†',  '1',  '"',  '/',  '5',  '4',  '0',  ';',  '¾',  '‘',  '¼',  '{',  ')',  '…',  '½',  '6',  '×',  '[',  '+',  ':',  '$',  '3',  '?',  '7',  '©',  '’',  '£',  '8']

pat_verb_noun_phrase =[
    {'POS': {'IN': 'VERB ADV'.split()}, 'OP':'+'}, 
    {'POS': {'IN' : 'DET ADP PART PRON'.split()}, 'OP':'*'},
    {'POS': {'IN': 'NOUN ADJ PROPN'.split()}, 'OP':'+'}
]


pat_noun_verb_phrase = [
            {'POS': {'IN': ['NOUN', 'ADJ']}, 'OP': '+'},
            {'POS': {'IN': ['AUX', 'PART', 'ADP']}, 'OP': '*'},
            {'POS': {'IN': ['VERB', 'ADV']}, 'OP': '+'}
            ]
            
            

pat_n_p_v = [
            {'POS': {'IN': ['NOUN', 'ADJ', 'PROPN']}, 'OP': '+'},
            {'POS': {'IN': ['PUNCT', 'CCONJ']}, 'LEMMA': {'NOT_IN': ignore_list}, 'OP': '+'},
            {'POS': {'IN': ['NOUN', 'ADJ', 'VERB', 'PROPN']}, 'OP':'+'}
             ]            
             
pat_noun = [
    {'POS': {'IN': 'NOUN ADJ PROPN'.split()} , 'OP':'+'}
                        ]

pat_n_n = [
    {'POS': {'IN': ['NOUN', 'ADJ', 'PROPN']}, 'OP': '+'},
    {'POS': {'IN': ['DET', 'ADP', 'PART']}, 'OP': '+'},
    {'POS': {'IN': ['NOUN', 'ADJ', 'PROPN']}, 'OP': '+'}
    ]

In [9]:
list_pos_patterns = [i for i in dir() if i.startswith('pat_')]

In [10]:
try:
    threshold_pattern = int(input('''
Enter threhold per pattern per batch:  '''))
    print(f"""
Threshold is {threshold_pattern}
    """)
except:
    print('Threshold must be an integer number; 2 was chosen for you')
    threshold_pattern = 2


Enter threhold per pattern per batch:   2



Threshold is 2
    


In [11]:
def get_pos_lemma_pattern(doc, pat_name='anything', pat_collection='list_of_dictionaries'):
    '''
    Given a spacy doc object; find the span according to the pattern given.
    ''' 
    matcher =  Matcher(vocab = nlp.vocab)
    matcher.add(f'{pat_name}', pat_collection, greedy='LONGEST')
    
    
    doc_match = matcher(doc)
    
    
    list_container = []
    for match in doc_match:
        start = match[1]
        end = match[2]
        
        result_lemma = doc[start:end]
        result_lemma = [(i.lemma_ + i.whitespace_) for i in result_lemma] # new!
        result_lemma = ''.join(result_lemma).strip()

        result_token = doc[start:end]
        result_token = [(i.text + i.whitespace_) for i in result_token] # new!
        result_token = ''.join(result_token).strip()

        result_pos = doc[start:end]
        result_pos = [i.pos_ for i in result_pos] # new!
        result_pos = ' '.join(result_pos).strip()
        
        if len(result_lemma.split()) > 1:
            list_container.append((result_token, result_lemma, result_pos))

    return list_container

In [13]:
#os.mkdir('IOC_GENERAL')

In [14]:
# for root, dirs, files in os.walk('NLP/'):
#     files = sorted(files)
#     files_path = [os.path.join(root, file) for file in files if file.endswith('.pkl')]
#     files_path = [f'./{file}' for file in files_path]

In [15]:
files_path = ['IOC_news_text.pkl']

In [19]:
for (idx, file) in enumerate(files_path):
    print(f'Processing {file} Batch {idx+1} of {len(files_path)}')

    df_temp = pd.read_pickle(file)
    
    
    print('Starting NLP')
    
    start_nlp = time_now()
    df_temp['NLP'] = df_temp['TEXT'].swifter.apply(lambda x: nlp(x))
    
    finish_nlp = time_now()
    print('Finished NLP')
    duration  = ((finish_nlp - start_nlp).seconds)/60
    print(f'Total NLP Duration: {duration} minutes')
    
    for pat in list_pos_patterns:
        pattern_name = pat.upper()
        df_temp[pattern_name] = df_temp['NLP'].swifter.apply(lambda x: get_pos_lemma_pattern(x, f'{pat}', [eval(pat)]))
        
        list_pat = list(chain(*df_temp[pattern_name]))
        df_pattern  = pd.DataFrame(data=list_pat)
        df_pattern = df_pattern.rename(columns={0: 'TEXT', 1: 'LEMMA', 2: 'POS'})
        
        dict_pattern = dict(df_pattern['LEMMA'].value_counts())
        dict_pattern = {k:v for k,v in dict_pattern.items() if v >= threshold_pattern}
        
        df_pattern = df_pattern[df_pattern['LEMMA'].swifter.apply(lambda x: x in dict_pattern)].reset_index(drop=True)
        df_pattern.to_pickle(F'./IOC_GENERAL/BATCH_{str(idx+1).zfill(2)}_{pattern_name}.pkl', protocol=4)
    print(f'Finished {file} Batch {idx+1} of {len(files_path)}')

Processing IOC_news_text.pkl Batch 1 of 1
Starting NLP
Current Time = 17:36:25.026660


Pandas Apply:   0%|          | 0/480405 [00:00<?, ?it/s]

Current Time = 18:57:12.310858
Finished NLP
Total NLP Duration: 80.78333333333333 minutes


Pandas Apply:   0%|          | 0/480405 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/512797 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/480405 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/371623 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/480405 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/892200 [00:00<?, ?it/s]

Finished IOC_news_text.pkl Batch 1 of 1


In [23]:
5

5

In [ ]:
test = pd.read_pickle('OUTPUT_GENDER/BATCH_01_PAT_N_N.pkl')

In [ ]:
test['LEMMA'].value_counts()

In [26]:
5

5

In [27]:
55

55

,TEXT,LEMMA,POS
0,provide the highlight,provide the highlight,VERB DET NOUN
1,struck gold,strike gold,VERB NOUN
2,produced a late burst,produce a late burst,VERB DET ADJ NOUN
3,assured of silver,assure of silver,VERB ADP NOUN
4,secure bronze,secure bronze,VERB NOUN
...,...,...,...
456830,win the next,win the next,VERB DET ADJ
456831,turned the tide,turn the tide,VERB DET NOUN
456832,exhausting rallies,exhaust rally,VERB NOUN
456833,reigning Olympic champion Viktor Axelsen,reign olympic champion Viktor Axelsen,VERB ADJ NOUN PROPN PROPN


In [30]:
dict_verbs_gen = dict(Counter(list(chain(*df_temp.NLP.apply(lambda x: [i.lemma_ for i in x if i.pos_ == 'VERB'])))).most_common())

In [31]:
dict_nouns_gen = dict(Counter(list(chain(*df_temp.NLP.apply(lambda x: [i.lemma_ for i in x if i.pos_ == 'NOUN'])))).most_common())

In [32]:
dict_adj_gen = dict(Counter(list(chain(*df_temp.NLP.apply(lambda x: [i.lemma_ for i in x if i.pos_ == 'ADJ'])))).most_common())

In [33]:
dict_adv_gen = dict(Counter(list(chain(*df_temp.NLP.apply(lambda x: [i.lemma_ for i in x if i.pos_ == 'ADV'])))).most_common())

In [34]:
pd.DataFrame(data = dict_adj_gen.items()).rename(columns={0: 'WORD', 1:'FREQ' }).to_pickle('IOC_GENERAL_ADJECTIVES.pkl', protocol=4)

In [35]:
pd.DataFrame(data = dict_adv_gen.items()).rename(columns={0: 'WORD', 1:'FREQ' }).to_pickle('IOC_GENERAL_ADVERBS.pkl', protocol=4)

In [36]:
pd.DataFrame(data = dict_nouns_gen.items()).rename(columns={0: 'WORD', 1:'FREQ' }).to_pickle('IOC_GENERAL_NOUNS.pkl', protocol=4)

In [37]:
pd.DataFrame(data = dict_verbs_gen.items()).rename(columns={0: 'WORD', 1:'FREQ' }).to_pickle('IOC_GENERAL_VERBS.pkl', protocol=4)

In [43]:
pd.read_pickle('IOC_GENERAL/BATCH_01_PAT_NOUN_VERB_PHRASE.pkl')

,TEXT,LEMMA,POS
0,late burst to secure,late burst to secure,ADJ NOUN PART VERB
1,silver already,silver already,NOUN ADV
2,bronze ahead,bronze ahead,NOUN ADV
3,Indian shooters had managed,indian shooter have manage,ADJ NOUN AUX VERB
4,unable to participate,unable to participate,ADJ PART VERB
...,...,...,...
170174,play resumed,play resume,NOUN VERB
170175,title shot,title shoot,NOUN VERB
170176,players went,player go,NOUN VERB
170177,guns blazing,gun blaze,NOUN VERB


OSError: [Errno 12] Cannot allocate memory